![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [218]:
import pandas as pd
import numpy as np

# Client CSV columns variable
df = pd.read_csv("bank_marketing.csv")

# Create client dataframe
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

# Column accuracy check
# print(client.head())

# Convert credit_defautl data type to boolean
client['credit_default'] = client['credit_default'].astype('bool')

# Check that the data type conversion was successful
assert client['credit_default'].dtype == 'bool'

# Convert mortgage data type to boolean
client['mortgage'] = client['mortgage'].astype('bool')

# Check that the data type conversion was successful
assert client['mortgage'].dtype == 'bool'

# Convert . to _ in the job column
client['job'] = client['job'].str.replace(".", "_")

# Convert . to _ in the education column
client['education'] = client['education'].str.replace(".", "_")

# Convert unknown to np.NaN in the education column
client['education'] = client['education'].replace("unknown", np.nan)

#print(client['job'].head())

#print(client['education'].head())

print(client.info())

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188

In [219]:
# Create campaign dataframe
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]

#print(campaign.info())

# Convert previous_outcome data type to boolean
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')

# Check that the data type conversion was successful
assert campaign['previous_outcome'].dtype == 'bool'

# Convert campaign_outcome data type to boolean
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

# Check that the data type conversion was successful
assert campaign['campaign_outcome'].dtype == 'bool'

# Create new column year with value of 2022
campaign['year'] = "2022"

# Convert day column into object
df['day'] = df['day'].astype(str)

assert df['day'].dtype == 'object'

# Create last_contact_date column
campaign['last_contact_date'] = campaign['year'] + '-' + df['month']+ '-' + df['day']

# Convert last_contact_date to datetime format
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], format='%Y-%b-%d')

#print(campaign.head())

print(campaign.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   year                        41188 non-null  object        
 7   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4), object(1)
memory usage: 2.0+ MB
None


In [220]:
import pandas as pd

economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]

print(economics.head())


   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857


In [221]:
# Create requested csv files
client.to_csv('client.csv')
campaign.to_csv('campaign.csv')
economics.to_csv('economics.csv')